In [ ]:
# Import necessary libraries
import streamlit as st
import bs4
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from dotenv import find_dotenv, load_dotenv

# Load environment variables from .env file
load_dotenv(find_dotenv())

# Define a function to format documents
def format_docs(docs):
    """
    Format a list of documents into a string.

    Args:
    docs (list): A list of documents.

    Returns:
    str: Formatted string of documents.
    """
    return "\n\n".join(doc.page_content for doc in docs)

# Initialize the DeepLake dataset
dat = 'hub://mpazaryna/hospital-room-products'
vectordb = DeepLake(dataset_path=dat, read_only=True, embedding=OpenAIEmbeddings())

# Initialize the retriever from vectorai
retriever = vectordb.as_retriever()

# Pull the prompt model from vectorhub
prompt = hub.pull("rlm/rag-prompt")

# Initialize the ChatOpenAI model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Streamlit app setup
def main():
    """
    Main function to run the Streamlit app.
    """
    # Set the title of the Streamlit app
    st.title('Your Streamlit App')

    # Create a text input field for user queries
    user_input = st.text_input("Enter your query:")

    # Process the input when a query is submitted
    if user_input:
        # Create the processing chain
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        # Execute the processing chain and get the response
        response = rag_chain(user_input)

        # Display the response on the app
        st.write(response)

# Ensure the script runs only when directly executed
if __name__ == "__main__":
    main()
